In [ ]:
import json
import time
from requests import get, post

# Endpoint URL
endpoint = r"https://<resourcename>.cognitiveservices.azure.com/"

#Source to the training data inside azure storage account i.e blob container
source = r"https://formrecognizestore.blob.core.windows.net/<containername>?<**********>&sig=<**********>
#subfolder name
prefix = "formblob"
#subfolder set to True
includeSubFolders = True
#Train custom form with labels 
useLabelFile = False

headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': '<******************>',
}

body =     {
    "source": source,
    "sourceFilter": {
        "prefix": prefix,
        "includeSubFolders": includeSubFolders
    },
    "useLabelFile": useLabelFile
}
#Request URI- https://{endpoint}/formrecognizer/v2.1-preview.2/custom/models
post_url = endpoint + r"/formrecognizer/v2.1-preview.2/custom/models"
try:
    resp = post(url = post_url, json = body, headers = headers)
    if resp.status_code != 201:
        print("POST model failed (%s):\n%s" % (resp.status_code, json.dumps(resp.json())))
        quit()
    print("POST model succeeded:\n%s" % resp.headers)
    get_url = resp.headers["location"]
except Exception as e:
    print("POST model failed:\n%s" % str(e)) #Printing the error message
    quit() 

In [ ]:
try:
    resp = get(url = get_url, headers = headers)
    resp_json = resp.json()
    if resp.status_code != 200:
        print("GET model failed (%s):\n%s" % (resp.status_code, json.dumps(resp_json)))
        quit()
    model_status = resp_json["modelInfo"]["status"]
    if model_status == "ready":
        print("Training succeeded:\n%s" % json.dumps(resp_json))
        
    if model_status == "invalid":
        print("Training failed. Model is invalid:\n%s" % json.dumps(resp_json))
        quit()
        # Training still running. Wait and retry.
        #time.sleep(wait_sec)
        #n_try += 1
        #wait_sec = min(2*wait_sec, max_wait_sec)     
except Exception as e:
    msg = "GET model failed:\n%s" % str(e)
    print(msg)
    quit()